# Exploritory Data Analysis

### Data cleaning

In [1]:
import json
import pandas as pd
import re

In [2]:
# import 2018 goodreads books
with open("Goodreads_books_2018.json") as datafile:
    data_2018 = json.load(datafile)

In [3]:
# import 2017 goodreads books
with open("Goodreads_books_2017.json") as datafile:
    data_2017 = json.load(datafile)

In [75]:
df_2018 = pd.DataFrame(data_2018)

In [76]:
df_2017 = pd.DataFrame(data_2017)

In [77]:
# join the dataframes
df_goodreads = pd.concat([df_2017,df_2018])

In [80]:
df_goodreads.reset_index(drop=True,inplace=True)

In [81]:
df_goodreads.shape

(2803, 10)

In [82]:
df_goodreads.index

RangeIndex(start=0, stop=2803, step=1)

In [83]:
df_goodreads = df_goodreads[['ISBN','title','series','author','rating','genre','publish_date','publish_company','format','number_of_pages']]

In [84]:
df_goodreads.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2803 entries, 0 to 2802
Data columns (total 10 columns):
ISBN               2803 non-null object
title              2803 non-null object
series             2803 non-null object
author             2803 non-null object
rating             2803 non-null object
genre              2803 non-null object
publish_date       2803 non-null object
publish_company    2803 non-null object
format             2803 non-null object
number_of_pages    2803 non-null object
dtypes: object(10)
memory usage: 219.1+ KB


In [85]:
# get rid of rows where ISBN wasn't actually an ISBN
# use a reg ex variable declaring that the ISBN must be 13 digits
reg_ex = r'^(\d{13})$'

df_goodreads = df_goodreads[df_goodreads['ISBN'].str.contains(reg_ex)]

/Users/Kristinabarounis/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """


In [86]:
# make series column binary (0 if not part of a series, 1 if it is)
df_goodreads['series'].loc[(df_goodreads['series'] == "Edit Details")] = 0
df_goodreads['series'].loc[(df_goodreads['series'] != 0)] = 1

In [87]:
df_goodreads['rating'] = df_goodreads['rating'].astype(float)

In [88]:
# split up the date into month and year
df_goodreads['publish_month'] = df_goodreads['publish_date'].str.split(n=2,expand = True)[0]
df_goodreads['publish_year'] = df_goodreads['publish_date'].str.split(n=2,expand = True)[2]

In [89]:
# 1172 books from goodreads between 2017 and 2018 bc we dropped the empty ISBNs
df_goodreads.shape

(1172, 12)

In [90]:
# reset the index again
df_goodreads.reset_index(drop=True, inplace=True)

In [91]:
df_goodreads.index

RangeIndex(start=0, stop=1172, step=1)

In [92]:
# need to drop duplicates
df_goodreads[df_goodreads.duplicated(subset='ISBN') == True]

,ISBN,title,series,author,rating,genre,publish_date,publish_company,format,number_of_pages,publish_month,publish_year
927,9780062676788,The Word Is Murder,1,Anthony Horowitz,3.92,mystery,June 5th 2018,Harper (first published August 24th 2017),Hardcover,390,June,2018


In [93]:
df_goodreads[df_goodreads['ISBN'] == '9780062676788']

,ISBN,title,series,author,rating,genre,publish_date,publish_company,format,number_of_pages,publish_month,publish_year
545,9780062676788,The Word Is Murder,1,Anthony Horowitz,3.92,mystery,June 5th 2018,Harper (first published August 24th 2017),Hardcover,390,June,2018
927,9780062676788,The Word Is Murder,1,Anthony Horowitz,3.92,mystery,June 5th 2018,Harper (first published August 24th 2017),Hardcover,390,June,2018


In [94]:
# let's drop the second one
df_goodreads = df_goodreads.drop(index = 545)

In [95]:
# now there's only one!
df_goodreads[df_goodreads['ISBN'] == '9780062676788']

,ISBN,title,series,author,rating,genre,publish_date,publish_company,format,number_of_pages,publish_month,publish_year
927,9780062676788,The Word Is Murder,1,Anthony Horowitz,3.92,mystery,June 5th 2018,Harper (first published August 24th 2017),Hardcover,390,June,2018


In [96]:
# import the NYT bestsellers
with open("list_of_NYT_books.json") as datafile:
    NYT_bestsellers = json.load(datafile)

In [216]:
df_NYT = pd.DataFrame(NYT_bestsellers)

In [217]:
# do everything to this dataframe that we did to the df above before merging

In [218]:
df_NYT = df_NYT[['ISBN','title','series','author','rating','genre','publish_date','publish_company','format','number_of_pages']]

In [219]:
# make series a binary value
df_NYT['series'].loc[(df_NYT['series'] == "Edit Details")] = 0
df_NYT['series'].loc[(df_NYT['series'] != 0)] = 1

In [220]:
# split up the date into month and year
df_NYT['publish_month'] = df_NYT['publish_date'].str.split(n=2,expand = True)[0]
df_NYT['publish_year'] = df_NYT['publish_date'].str.split(n=2,expand = True)[2]

In [221]:
# convert rating to a float
df_NYT['rating'] = df_NYT['rating'].astype(float)

In [222]:
# set ISBN as unique index value
# df_NYT.set_index('ISBN',inplace=True)

In [223]:
# create a column of 1's to represent that these are all bestsellers
df_NYT['NYT_bestseller'] = 1

In [224]:
len(df_NYT)

741

In [225]:
df_NYT = df_NYT.drop_duplicates()

In [226]:
len(df_NYT)

558

In [227]:
# df_NYT = df_NYT.loc[~df_NYT.index.duplicated(keep='first')]

In [228]:
full_df = pd.concat([df_NYT,df_goodreads], sort=False)

In [229]:
full_df.shape

(1729, 13)

In [230]:
# let's reset the index
full_df.reset_index(drop=True, inplace=True)

In [231]:
full_df.index

RangeIndex(start=0, stop=1729, step=1)

In [232]:
# fill in the non-bestsellers with 0 in that column
full_df['NYT_bestseller'].fillna(value=0,inplace=True)

In [233]:
# look at dates that weren't coded properly
full_df[(full_df['publish_month'] != 'January') &
        (full_df['publish_month'] != 'February') &
        (full_df['publish_month'] != 'March') &
        (full_df['publish_month'] != 'April') &
        (full_df['publish_month'] != 'May') &
        (full_df['publish_month'] != 'June') &
        (full_df['publish_month'] != 'July') &
        (full_df['publish_month'] != 'August') &
        (full_df['publish_month'] != 'September') &
        (full_df['publish_month'] != 'October') &
        (full_df['publish_month'] != 'November') &
        (full_df['publish_month'] != 'December')]


,ISBN,title,series,author,rating,genre,publish_date,publish_company,format,number_of_pages,publish_month,publish_year,NYT_bestseller
438,9780399592805,The Golden House,0,Salman Rushdie,3.66,fiction,2017,Random House,Hardcover,380,2017,None,1.0
1035,9781927990216,Prime Meridian,0,Silvia Moreno-Garcia,4.03,science-fiction,,Innsmouth Free Press (first published December...,Paperback,122,None,None,0.0
1381,9789381841372,Wedding Pickle,0,Neha Sharma,4.03,asian-literature,2018,Grapevine India,Paperback,264,2018,None,0.0
1597,9780300219333,Packing My Library: An Elegy and Ten Digressi...,0,Alberto Manguel,3.88,writing,2018,Yale University Press,Hardcover,146,2018,None,0.0
1692,9780062468741,The Iron Season,1,Helene Wecker,4.03,fantasy,Expected publication: 2020,Harper,ebook,448,Expected,2020,0.0


In [234]:
# manually fix these, except the last one which hasn't been published yet - let's drop that one
full_df['publish_month'].loc[full_df['ISBN'] == '9780399592805'] = 'September'
full_df['publish_month'].loc[full_df['ISBN'] == '9781927990216'] = 'July'
full_df['publish_month'].loc[full_df['ISBN'] == '9789381841372'] = 'January'
full_df['publish_month'].loc[full_df['ISBN'] == '9780300219333'] = 'March'
full_df = full_df.drop(index=1141)

In [235]:
# lets also fix the years
full_df['publish_year'].loc[full_df['ISBN'] == '9780399592805'] = '2017'
full_df['publish_year'].loc[full_df['ISBN'] == '9781927990216'] = '2018'
full_df['publish_year'].loc[full_df['ISBN'] == '9789381841372'] = '2018'
full_df['publish_year'].loc[full_df['ISBN'] == '9780300219333'] = '2018'

In [236]:
# lets do the same thing to check that the year was coded correctly but use regex since 
# so we don't have to manually write out all the years
reg_ex_year = r'^(\s*2[0-9]{3}\s*)$' 

full_df[full_df['publish_year'].str.contains(reg_ex_year) == False]

/Users/Kristinabarounis/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """


,ISBN,title,series,author,rating,genre,publish_date,publish_company,format,number_of_pages,publish_month,publish_year,NYT_bestseller
398,9780061122415,The Alchemist,0,Paulo Coelho,3.85,fiction,May 1st 1993,HarperCollins (first published 1988),Paperback,197,May,1993,1.0
880,9781520356686,Fire in His Blood,1,Ruby Dixon,3.82,fantasy,January 11th 2017(first published January 9th ...,,Paperback,331,January,2017(first published January 9th 2017),0.0
1126,9781946836038,Amnesia,1,Cambria Hebert,3.99,romance,June 13th 2017(first published June 10th 2017),,Paperback,408,June,2017(first published June 10th 2017),0.0


In [237]:
# manually change the last two (the first one is correct just written a long time ago!)
full_df['publish_year'].loc[full_df['ISBN'] == '9781520356686'] = '2017'
full_df['publish_year'].loc[full_df['ISBN'] == '9781946836038'] = '2017'

In [238]:
# all fixed!

full_df[full_df['publish_year'].str.contains(reg_ex_year) == False]

/Users/Kristinabarounis/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  This is separate from the ipykernel package so we can avoid doing imports until


,ISBN,title,series,author,rating,genre,publish_date,publish_company,format,number_of_pages,publish_month,publish_year,NYT_bestseller
398,9780061122415,The Alchemist,0,Paulo Coelho,3.85,fiction,May 1st 1993,HarperCollins (first published 1988),Paperback,197,May,1993,1.0


In [239]:
# there are cases where some books are duplicated (58)- the one's shown below are from goodreads
# and are encoded as not best sellers, evne though they really are since they also appear
# in the NYT dataframe
len(full_df[full_df.duplicated(subset='ISBN') == True])

58

In [240]:
# make the duplicates a 1 in the NYT best seller column so they are TRULY the same rows
# that way we can use duplicated to drop them and the result will be one copy
full_df['NYT_bestseller'].loc[full_df.duplicated(subset='ISBN')==True] = 1

In [241]:
full_df=full_df.drop_duplicates()

In [242]:
full_df.shape

(1671, 13)

In [243]:
full_df.reset_index(drop=True, inplace=True)

In [244]:
full_df.index

RangeIndex(start=0, stop=1671, step=1)

In [245]:
# all the titles have extra spaces - oops! let's fix that!
full_df['title'] =[x.strip() for x in full_df['title']]

In [246]:
# create new feature for number of characters in a title
full_df['length_of_title'] = [len(x) for x in full_df['title']]

In [247]:
# create new feature for number of words in a title
full_df['num_words_in_title'] = [len(x.split()) for x in full_df['title']]

In [248]:
# because of how we scraped, there are null values in the df currently coded as strings
import numpy as np
np.where(full_df.applymap(lambda x: x == ''))

(array([  60,  120,  133,  133,  292,  340,  440,  530,  546,  557,  585,
         632,  845,  860,  906,  909,  915,  932,  967,  969,  996, 1010,
        1010, 1010, 1024, 1040, 1063, 1076, 1077, 1083, 1094, 1302, 1365,
        1365, 1488, 1502, 1544, 1585, 1611, 1626]),
 array([9, 9, 5, 9, 9, 9, 8, 9, 9, 9, 7, 7, 7, 9, 9, 5, 5, 5, 7, 9, 6, 5,
        7, 9, 7, 9, 7, 7, 9, 9, 7, 7, 7, 9, 5, 9, 9, 9, 9, 5]))

In [249]:
# let's replace them with nan values 
full_df.replace(r'^\s*$', np.nan, regex=True, inplace = True)

In [250]:
# now they are gone
np.where(full_df.applymap(lambda x: x == ''))

(array([], dtype=int64), array([], dtype=int64))

In [251]:
# let's see the empty values now
full_df.isna().sum()

ISBN                   0
title                  0
series                 0
author                 0
rating                 0
genre                  7
publish_date           1
publish_company       11
format                 1
number_of_pages       20
publish_month          0
publish_year           5
NYT_bestseller         0
length_of_title        0
num_words_in_title     0
dtype: int64

In [252]:
# deal with empty values
full_df[full_df['publish_date'].isna()]
# this is fine we manually fixed the month and year columns already

,ISBN,title,series,author,rating,genre,publish_date,publish_company,format,number_of_pages,publish_month,publish_year,NYT_bestseller,length_of_title,num_words_in_title
996,9781927990216,Prime Meridian,0,Silvia Moreno-Garcia,4.03,science-fiction,NaN,Innsmouth Free Press (first published December...,Paperback,122,July,2018,0.0,14,2


In [253]:
full_df[full_df['publish_year'].isna()]

,ISBN,title,series,author,rating,genre,publish_date,publish_company,format,number_of_pages,publish_month,publish_year,NYT_bestseller,length_of_title,num_words_in_title
26,9780061792663,American Gods,1,Neil Gaiman,4.11,fantasy,June 2001,HarperCollins e-books,ebook,541,June,None,1.0,13,2
436,9781594633669,The Girl on the Train,0,Paula Hawkins,3.91,fiction,April 2019,Riverhead Books (first published January 13th ...,Hardcover,336,April,None,1.0,21,5
690,9781760293512,Frogkisser!,0,Garth Nix,3.77,fantasy,March 2017,Allen & Unwin (first published February 28th 2...,Paperback,336,March,None,0.0,11,1
1002,9781612196015,Why I Am Not a Feminist: A Feminist Manifesto,0,Jessa Crispin,3.54,feminism,February 2017,Melville House,Paperback,151,February,None,0.0,45,9
1101,9781335014900,As I Am,1,A.M. Arthur,4.03,romance,March 2017,Carina Press (first published February 13th 2017),Paperback,368,March,None,0.0,7,3


In [254]:
# manually fix these

full_df['publish_year'].loc[full_df['ISBN'] == '9780061792663'] = '2001'
full_df['publish_year'].loc[full_df['ISBN'] == '9781594633669'] = '2019'
full_df['publish_year'].loc[full_df['ISBN'] == '9781760293512'] = '2017'
full_df['publish_year'].loc[full_df['ISBN'] == '9781612196015'] = '2017'
full_df['publish_year'].loc[full_df['ISBN'] == '9781335014900'] = '2017'

In [256]:
# let's drop these since genre is missing and the genre reflected in the rest of teh datafarme
# was taken from goodreads and based on peoples votes - genre is subjective (for our data)
full_df[full_df['genre'].isna()]

,ISBN,title,series,author,rating,genre,publish_date,publish_company,format,number_of_pages,publish_month,publish_year,NYT_bestseller,length_of_title,num_words_in_title
133,9781621576785,How to Achieve,0,Vince M Bertram,3.50,NaN,August 1st 2017,Regnery Publishing,Hardcover,NaN,August,2017,1.0,14,3
909,9781912551118,Walk Quiet Run Quick,0,Jacqueline Francis,2.50,NaN,March 3rd 2018,Conscious Dreams Publishing (first published 2...,Paperback,164,March,2018,0.0,20,4
915,9781522072942,The Hackers Code: The Great Prison Escape Proj...,0,A.K. Mars,4.67,NaN,August 13th 2017,A. K. MARS,Paperback,288,August,2017,0.0,78,15
932,9781978493162,"New Trump Order: Get In, Fit in or Step Aside",0,America Speaks,5.00,NaN,October 19th 2017,Createspace,Paperback,266,October,2017,0.0,45,10
1010,9781897913949,Incorruption,0,Jack McGinnigle,4.00,NaN,February 7th 2017,NaN,Paperback,NaN,February,2017,0.0,12,1
1488,9782021377613,Les vacances du petit Renard,0,Arthur Cahn,3.29,NaN,January 4th 2018,Le Seuil,Kindle Edition,192,January,2018,0.0,28,5
1626,9781725040304,Knitting in the City Coloring Book,1,Penny Reid,4.82,NaN,September 17th 2018,CreateSpace (first published September 11th 2018),Paperback,66,September,2018,0.0,34,6


In [263]:
full_df.dropna(subset=['genre'], inplace=True)

In [264]:
# they're gone!
full_df[full_df['genre'].isna()]

,ISBN,title,series,author,rating,genre,publish_date,publish_company,format,number_of_pages,publish_month,publish_year,NYT_bestseller,length_of_title,num_words_in_title


### Data exploration

In [ ]:
pd.crosstab(full_df.genre, full_df.NYT_bestseller, normalize='index')

# Modeling

### Model 1: Logistic Regression

### Model 2: Decision Tree

### Model 3: Random Forest

### Model 4: XGBoost

### Grid search - tuning hyperparamters